In [1]:
import os 
os.environ['REQUESTS_CA_BUNDLE'] = '/Users/XXXXXX/MSPref/Transformers/hf.pem'

In [2]:
## Prepare a basic Hugging face pipeline for NER using dbmdz/bert-large-cased-finetuned-conll03-english 
##  24 x BertLayer - Total params: 332,538,889 
## dslim/bert-base-NER is fine tuned on https://huggingface.co/datasets/conll2003

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import time
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#nlp = pipeline("ner", model=model, tokenizer=tokenizer , aggregation_strategy="simple")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
nlp("My name is Sylvain and I work at Hugging Face in Brooklyn.")
#nlp("LONDON 1996-08-30")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.99938285,
  'index': 4,
  'word': 'S',
  'start': 11,
  'end': 12},
 {'entity': 'I-PER',
  'score': 0.99815494,
  'index': 5,
  'word': '##yl',
  'start': 12,
  'end': 14},
 {'entity': 'I-PER',
  'score': 0.99590707,
  'index': 6,
  'word': '##va',
  'start': 14,
  'end': 16},
 {'entity': 'I-PER',
  'score': 0.99923277,
  'index': 7,
  'word': '##in',
  'start': 16,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.9738931,
  'index': 12,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976115,
  'index': 13,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9887976,
  'index': 14,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9932105,
  'index': 16,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [4]:
# Takes input record - '11\tCRICKET\t-\tLEICESTERSHIRE\tTAKE\tOVER\tAT\tTOP\tAFTER\tINNINGS\tVICTORY\t.\t0\t0\t3\t0\t0\t0\t0\t0\t0\t0\t0\n'
# Truncate new line and split by tab and add whitespace.

In [5]:
def map_record_to_infer_data(record):
    record = record.strip()
    record = record.split("\t")
    #record = tf.strings.split(record, sep="\t")
    length = int(record[0])
    tokens = record[1 : length + 1]
    pri_sentence = ""
    for tk in tokens:
        pri_sentence += tk
        pri_sentence += " "
    pri_sentence = pri_sentence.strip()
    tags = record[length + 1 :]
    #tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags = [int(i) for i in tags]
    #tags += 1
    return pri_sentence, tags

In [6]:
val_data_file="/Users/XXXXXX/MSPref/Transformers/LLM/conlldata/conll_val.txt"
f=open(val_data_file)
val_data=f.readlines()
f.close()

In [7]:
val_data[0]

'11\tCRICKET\t-\tLEICESTERSHIRE\tTAKE\tOVER\tAT\tTOP\tAFTER\tINNINGS\tVICTORY\t.\t0\t0\t3\t0\t0\t0\t0\t0\t0\t0\t0\n'

In [8]:
def make_tag_lookup_table():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    #all_labels = ["[PAD]"] + all_labels
    return dict(zip(range(0, len(all_labels) + 1), all_labels))


nermapping = make_tag_lookup_table()
print(nermapping)

{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}


In [9]:
nerk = dict((v,k) for k,v in nermapping.items())
print(nerk)

{'[PAD]': 0, 'O': 1, 'B-PER': 2, 'I-PER': 3, 'B-ORG': 4, 'I-ORG': 5, 'B-LOC': 6, 'I-LOC': 7, 'B-MISC': 8, 'I-MISC': 9}


In [10]:
def return_pridict_ner_map(sns,priout):
    #print(sns)
    #print(priout)
    snsa = sns.split(" ")
    priidx=[]
    for si in snsa:
        priidx.append(0)
    for ners in priout:
        #print(ners)
        if 'index' in ners.keys() :
            #print(ners)
            if ners['word'] in snsa :
                #idx=ners['index'] - 1 
                idx=snsa.index(ners['word'])
                #priidx[idx]= nerk[ners['entity']]
                priidx[idx]= nerk[ners['entity']] - 1
    #print(priidx)
    return priidx

In [11]:
from torchinfo import summary
print(model)
summary(model)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 

Layer (type:depth-idx)                                  Param #
BertForTokenClassification                              --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              29,691,904
│    │    └─Embedding: 3-2                              524,288
│    │    └─Embedding: 3-3                              2,048
│    │    └─LayerNorm: 3-4                              2,048
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             302,309,376
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           9,225
Total params: 332,538,889
Trainable params: 332,538,889
Non-trainable params: 0

In [12]:
from conlleval import evaluate
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
def calculate_metrics(dataset):
    all_true_tag_ids, all_predicted_tag_ids = [], []
    for recp in dataset:
        x,y = map_record_to_infer_data(recp)
        #inputs = tokenizer(x, return_tensors="pt")
        #outputs = model(**inputs)
        #probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
        #predict = outputs.logits.argmax(dim=-1)[0].tolist()
        output = nlp(x)
        #output = getNERCL(x,predict, probabilities)
        #print(output)
        #=====
        true_tag_ids = np.reshape(y, [-1])
        predictions=return_pridict_ner_map(x,output)
        #print(x)
        #print(y)
        #print(predictions)
        predictions=np.reshape(predictions, [-1])
        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]
        #print(true_tag_ids)
        #print(predicted_tag_ids)
        #====
        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)
    #if len(all_true_tag_ids) == len(all_predicted_tag_ids) :
    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)
    predicted_tags = [nermapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [nermapping[tag] for tag in all_true_tag_ids]
        #print(unique(real_tags))
        #print(unique(predicted_tags))
    evaluate(real_tags, predicted_tags)
    #else :
        #print(len(all_true_tag_ids))
        #print(len(all_predicted_tag_ids))
start = time.time()
calculate_metrics(val_data)
end = time.time()
print("Total time taken ", (end - start))

processed 4748 tokens with 2879 phrases; found: 4748 phrases; correct: 1180.
accuracy:  34.88%; (non-O)
accuracy:  27.86%; precision:  24.85%; recall:  40.99%; FB1:  30.94
              LOC: precision:  11.52%; recall:  24.03%; FB1:  15.58  1345
             MISC: precision:  25.62%; recall:  95.59%; FB1:  40.41  847
              ORG: precision:  38.26%; recall:  37.98%; FB1:  38.12  1218
              PER: precision:  25.56%; recall:  43.85%; FB1:  32.29  1338
Total time taken  355.6348478794098
